# RAG-Enhanced ESG Scoring

This notebook implements a Retrieval-Augmented Generation (RAG) approach for ESG scoring, 
combining the methodologies from the Exercise folder with ESG analysis to overcome token limitations.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import json
import re
import unicodedata
from collections import defaultdict
from typing import List, Dict, Any
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# RAG-specific imports (from Exercise folder)
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings
from ibm_watsonx_ai.metanames import GenParams

# Evaluation imports
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Environment setup
from decouple import config
from dotenv import load_dotenv

print("RAG-Enhanced ESG Analysis Setup Complete")
print("=" * 50)

In [ ]:
# Load environment variables
env_path = "/Users/wenlong/Documents/GitHub/ma2/assignments/.env"
if os.path.exists(env_path):
    load_dotenv(dotenv_path=env_path)

# Get API credentials
WX_API_KEY = os.getenv("WX_API_KEY")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"
WX_PROJECT_ID = "fb3ce137-1a1e-411b-b5f4-d66b00a139f0"

if not WX_API_KEY:
    print("Warning: WX_API_KEY not found in environment variables")
else:
    print("✓ Environment variables loaded successfully")

In [ ]:
# Text preprocessing function (from Exercise/rag.ipynb)
def preprocess_text(text: str) -> str:
    """
    Basic document preprocessing:
    - Normalize Unicode characters
    - Remove HTML tags
    - Remove unwanted special characters
    - Normalize whitespace
    - Convert to lowercase
    """
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"<[^>]+>", " ", text)  # Remove HTML
    text = re.sub(r"[^a-zA-Z0-9.,;:!?()\-\'\"\n ]+", " ", text)  # Clean special chars
    text = re.sub(r"\s+", " ", text)  # Normalize whitespace
    return text.strip().lower()

def update_documents_with_headers(chunks):
    """
    Creates a new list of Document objects with page_content prepended with headers
    in [Header1/Header2/Header3]: format
    """
    updated_chunks = []
    
    for doc in chunks:
        # Create a deep copy of the document to avoid modifying the original
        new_doc = deepcopy(doc)
        
        # Get all headers that exist in metadata
        headers = []
        for i in range(1, 4):
            key = f'Header {i}'
            if key in new_doc.metadata:
                headers.append(new_doc.metadata[key])
        
        # Create the header prefix and update page_content
        if headers:
            prefix = f"[{'/'.join(headers)}]: "
            new_doc.page_content = prefix + "\n" + new_doc.page_content
        
        updated_chunks.append(new_doc)
    
    return updated_chunks

print("✓ Text preprocessing functions loaded")

In [ ]:
# Load benchmark ESG scores
def load_benchmark_scores():
    """Load benchmark ESG scores from benchMark.md"""
    benchmark_scores = {
        'Nike': 70, 'Apple': 70, 'Boeing': 79, 'Disney': 68, 'Eli Lilly': 64, 'FedEx': 71,
        'Johnson & Johnson': 85, 'JPMorgan Chase': 80, 'McDonald\'s': 66, 'Meta': 60,
        'Microsoft': 87, 'Netflix': 37, 'NVIDIA': 77, 'Tesla': 72, 'Google': 81
    }
    return benchmark_scores

# Load and preprocess company documents
def load_and_process_company_documents():
    """Load all ESG-related documents for each company and prepare for RAG"""
    data_path = "data"
    company_documents = {}
    
    # Company directory mapping
    company_dirs = {
        'Nike': 'nike', 'Apple': 'apple', 'Boeing': 'boeing', 'Disney': 'disney',
        'Eli Lilly': 'elililly', 'FedEx': 'fedex', 'Johnson & Johnson': 'johnsonandjohnson',
        'JPMorgan Chase': 'jpmorganchase', 'McDonald\'s': 'mcdonald', 'Meta': 'meta',
        'Microsoft': 'microsoft', 'Netflix': 'netflix', 'NVIDIA': 'nvidia',
        'Tesla': 'tesla', 'Google': 'google'
    }
    
    print("Loading and preprocessing company documents...")
    for company, dir_name in company_dirs.items():
        company_path = os.path.join(data_path, dir_name)
        if os.path.exists(company_path):
            documents = []
            file_count = 0
            
            # Load all relevant files
            for file in os.listdir(company_path):
                if file.endswith(('.md', '.txt')):
                    file_path = os.path.join(company_path, file)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                            
                        # Preprocess the content
                        cleaned_content = preprocess_text(content)
                        
                        # Create document object
                        doc = Document(
                            page_content=cleaned_content,
                            metadata={
                                "company": company,
                                "source_file": file,
                                "file_path": file_path
                            }
                        )
                        documents.append(doc)
                        file_count += 1
                        
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")
            
            if documents:
                company_documents[company] = documents
                total_chars = sum(len(doc.page_content) for doc in documents)
                print(f"✓ {company}: {file_count} files, {total_chars} characters")
            else:
                print(f"✗ {company}: No documents found")
        else:
            print(f"✗ {company}: Directory not found - {company_path}")
    
    return company_documents

# Load data
benchmark_scores = load_benchmark_scores()
company_documents = load_and_process_company_documents()

print(f"\nLoaded documents for {len(company_documents)} companies")
print(f"Benchmark scores available for {len(benchmark_scores)} companies")

In [ ]:
# Initialize LLM and embeddings (following Exercise methodology)
def setup_rag_components():
    """Setup LLM and embedding models for RAG"""
    
    # Initialize LLM with optimized parameters for ESG scoring
    llm = WatsonxLLM(
        model_id="ibm/granite-13b-instruct-v2",
        url=WX_API_URL,
        apikey=WX_API_KEY,
        project_id=WX_PROJECT_ID,
        params={
            GenParams.DECODING_METHOD: "greedy",
            GenParams.TEMPERATURE: 0,
            GenParams.MIN_NEW_TOKENS: 10,
            GenParams.MAX_NEW_TOKENS: 500,
            GenParams.REPETITION_PENALTY: 1.2
        }
    )
    
    # Initialize embeddings
    embeddings = WatsonxEmbeddings(
        model_id="ibm/granite-embedding-278m-multilingual",
        url=WX_API_URL,
        project_id=WX_PROJECT_ID,
        apikey=WX_API_KEY,
        params={}
    )
    
    return llm, embeddings

# Setup components
llm, embeddings = setup_rag_components()
print("✓ LLM and embeddings initialized")

In [ ]:
# Document chunking and vector store creation
def create_company_vector_stores(company_documents, embeddings):
    """Create vector stores for each company's documents"""
    
    company_retrievers = {}
    
    # Headers for markdown splitting (if needed)
    headers_to_split_on = [
        ("#", "Header 1"), 
        ("##", "Header 2"), 
        ("###", "Header 3"), 
        ("####", "Header 4")
    ]
    
    # Text splitter with semantic-aware separators
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=60,
        separators=["\n\n", "\n", ".", " ", ""],
    )
    
    print("Creating vector stores for each company...")
    
    for company, documents in company_documents.items():
        try:
            # Combine all documents for the company
            all_chunks = []
            
            for doc in documents:
                # Split document into chunks
                chunks = text_splitter.split_documents([doc])
                all_chunks.extend(chunks)
            
            if all_chunks:
                # Create vector store for this company
                vector_store = Chroma.from_documents(
                    collection_name=f"esg_{company.lower().replace(' ', '_').replace('&', 'and')}",
                    embedding=embeddings,
                    persist_directory=f"esg_vector_db_{company.lower().replace(' ', '_').replace('&', 'and')}",
                    documents=all_chunks,
                )
                
                # Create retriever
                retriever = vector_store.as_retriever(
                    search_type="similarity",
                    search_kwargs={"k": 5}  # Retrieve top 5 relevant chunks
                )
                
                company_retrievers[company] = retriever
                print(f"✓ {company}: {len(all_chunks)} chunks, vector store created")
            else:
                print(f"✗ {company}: No chunks created")
                
        except Exception as e:
            print(f"✗ {company}: Error creating vector store - {e}")
    
    return company_retrievers

# Create vector stores
company_retrievers = create_company_vector_stores(company_documents, embeddings)
print(f"\nCreated vector stores for {len(company_retrievers)} companies")

In [ ]:
# ESG scoring prompt template
def create_esg_scoring_template():
    """Create prompt template for ESG scoring"""
    
    template = """You are an expert ESG (Environmental, Social, Governance) analyst. Based on the provided company documents, analyze and score the company's ESG performance on a scale of 0-100.

Consider the following ESG factors:

ENVIRONMENTAL (0-35 points):
- Climate change initiatives and carbon footprint reduction
- Resource efficiency and waste management
- Environmental compliance and sustainability practices
- Renewable energy adoption and environmental innovation

SOCIAL (0-35 points):
- Employee wellbeing, diversity, and inclusion
- Community engagement and social impact
- Product safety and customer satisfaction
- Labor practices and human rights

GOVERNANCE (0-30 points):
- Board composition and independence
- Executive compensation and transparency
- Risk management and compliance
- Stakeholder engagement and business ethics

Company: {company}

Relevant Documents:
{context}

Based on the above information, provide:
1. Environmental score (0-35): [score] - [brief justification]
2. Social score (0-35): [score] - [brief justification]
3. Governance score (0-30): [score] - [brief justification]
4. Total ESG Score: [sum of above scores]

Only use information from the provided documents. If information is insufficient for any category, provide a conservative estimate and note the limitation.

ESG Analysis:"""
    
    return PromptTemplate.from_template(template)

# Create prompt template
esg_prompt = create_esg_scoring_template()
print("✓ ESG scoring prompt template created")

In [ ]:
# RAG-based ESG scoring function
def score_company_esg_rag(company, retriever, llm, prompt_template):
    """Score a company's ESG performance using RAG approach"""
    
    try:
        # Retrieve relevant documents for ESG analysis
        esg_query = f"ESG environmental social governance sustainability {company} climate diversity board ethics"
        retrieved_docs = retriever.invoke(esg_query)
        
        # Combine retrieved documents
        context = "\n\n".join([
            f"Document {i+1}:\n{doc.page_content}" 
            for i, doc in enumerate(retrieved_docs)
        ])
        
        # Format prompt
        formatted_prompt = prompt_template.invoke({
            "company": company,
            "context": context
        })
        
        # Generate ESG analysis
        response = llm.invoke(formatted_prompt)
        
        # Extract total score from response
        score = extract_esg_score(response)
        
        return {
            'company': company,
            'predicted_score': score,
            'analysis': response,
            'retrieved_docs_count': len(retrieved_docs)
        }
        
    except Exception as e:
        print(f"Error scoring {company}: {e}")
        return {
            'company': company,
            'predicted_score': 50,  # Default middle score
            'analysis': f"Error in analysis: {e}",
            'retrieved_docs_count': 0
        }

def extract_esg_score(response_text):
    """Extract the total ESG score from LLM response"""
    
    # Look for "Total ESG Score:" pattern
    patterns = [
        r"Total ESG Score:\s*(\d+)",
        r"Total:\s*(\d+)",
        r"Overall Score:\s*(\d+)",
        r"Final Score:\s*(\d+)",
        r"Score:\s*(\d+)"
    ]
    
    for pattern in patterns:
        match = re.search(pattern, response_text, re.IGNORECASE)
        if match:
            score = int(match.group(1))
            # Ensure score is within valid range
            return max(0, min(100, score))
    
    # Fallback: look for any number between 0-100
    numbers = re.findall(r'\b(\d{1,2}|100)\b', response_text)
    if numbers:
        # Take the last number found (often the total)
        return int(numbers[-1])
    
    # Default if no score found
    return 50

print("✓ RAG-based ESG scoring functions defined")

In [ ]:
# Run RAG-based ESG scoring for all companies
def run_rag_esg_analysis():
    """Run RAG-based ESG analysis for all companies"""
    
    results = []
    
    print("Running RAG-based ESG analysis...")
    print("=" * 50)
    
    # Process companies that have both retrievers and benchmark scores
    companies_to_analyze = set(company_retrievers.keys()) & set(benchmark_scores.keys())
    
    for company in tqdm(companies_to_analyze, desc="Analyzing companies"):
        print(f"\nAnalyzing {company}...")
        
        # Get retriever for this company
        retriever = company_retrievers[company]
        
        # Score using RAG approach
        result = score_company_esg_rag(company, retriever, llm, esg_prompt)
        
        # Add benchmark score
        result['benchmark_score'] = benchmark_scores[company]
        
        results.append(result)
        
        print(f"✓ {company}: Predicted={result['predicted_score']}, Benchmark={result['benchmark_score']}")
    
    return results

# Run the analysis
rag_results = run_rag_esg_analysis()
print(f"\nCompleted RAG analysis for {len(rag_results)} companies")

In [ ]:
# Evaluate RAG performance
def evaluate_rag_performance(results):
    """Evaluate the performance of RAG-based ESG scoring"""
    
    # Extract predictions and actual scores
    y_true = [r['benchmark_score'] for r in results]
    y_pred = [r['predicted_score'] for r in results]
    companies = [r['company'] for r in results]
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'Company': companies,
        'Benchmark_Score': y_true,
        'RAG_Predicted_Score': y_pred,
        'Absolute_Error': [abs(t - p) for t, p in zip(y_true, y_pred)]
    })
    
    print("RAG-Enhanced ESG Scoring Results")
    print("=" * 40)
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
    print(f"R² Score: {r2:.3f}")
    print(f"Mean Benchmark Score: {np.mean(y_true):.1f}")
    print(f"Mean Predicted Score: {np.mean(y_pred):.1f}")
    
    return results_df, {'MAE': mae, 'RMSE': rmse, 'R2': r2}

# Evaluate performance
results_df, metrics = evaluate_rag_performance(rag_results)

# Display detailed results
print("\nDetailed Results:")
print(results_df.round(2))

In [ ]:
# Visualize RAG performance
def visualize_rag_performance(results_df):
    """Create visualizations for RAG performance"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Predicted vs Actual scatter plot
    axes[0, 0].scatter(results_df['Benchmark_Score'], results_df['RAG_Predicted_Score'], 
                      alpha=0.7, s=100, color='blue')
    axes[0, 0].plot([30, 90], [30, 90], 'r--', label='Perfect Prediction')
    axes[0, 0].set_xlabel('Benchmark ESG Score')
    axes[0, 0].set_ylabel('RAG Predicted Score')
    axes[0, 0].set_title('RAG: Predicted vs Actual ESG Scores')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Error distribution
    axes[0, 1].hist(results_df['Absolute_Error'], bins=8, alpha=0.7, color='green', edgecolor='black')
    axes[0, 1].set_xlabel('Absolute Error')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Absolute Errors')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Company comparison
    x_pos = range(len(results_df))
    width = 0.35
    
    axes[1, 0].bar([x - width/2 for x in x_pos], results_df['Benchmark_Score'], 
                   width, label='Benchmark', alpha=0.7, color='orange')
    axes[1, 0].bar([x + width/2 for x in x_pos], results_df['RAG_Predicted_Score'], 
                   width, label='RAG Predicted', alpha=0.7, color='blue')
    
    axes[1, 0].set_xlabel('Companies')
    axes[1, 0].set_ylabel('ESG Score')
    axes[1, 0].set_title('Company-wise Score Comparison')
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(results_df['Company'], rotation=45, ha='right')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Error by company
    colors = ['red' if err > 15 else 'orange' if err > 10 else 'green' for err in results_df['Absolute_Error']]
    axes[1, 1].bar(range(len(results_df)), results_df['Absolute_Error'], 
                   color=colors, alpha=0.7)
    axes[1, 1].set_xlabel('Companies')
    axes[1, 1].set_ylabel('Absolute Error')
    axes[1, 1].set_title('Prediction Error by Company')
    axes[1, 1].set_xticks(range(len(results_df)))
    axes[1, 1].set_xticklabels(results_df['Company'], rotation=45, ha='right')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create visualizations
visualize_rag_performance(results_df)

In [ ]:
# Compare RAG performance with previous approaches
def compare_with_previous_results():
    """Compare RAG results with BOW, BERT, and standard LLM approaches"""
    
    # Previous results from the exercise (approximate values from exam report)
    previous_results = {
        'BOW': {'MAE': 4.65, 'RMSE': 6.12, 'R2': 0.89},
        'BERT': {'MAE': 6.30, 'RMSE': 8.45, 'R2': 0.82},
        'LLM (Standard)': {'MAE': 21.87, 'RMSE': 24.32, 'R2': 0.15},
        'RAG-LLM': metrics
    }
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(previous_results).T
    
    print("Performance Comparison Across All Approaches")
    print("=" * 50)
    print(comparison_df.round(3))
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    methods = comparison_df.index.tolist()
    
    # MAE comparison
    axes[0].bar(methods, comparison_df['MAE'], color=['skyblue', 'lightgreen', 'salmon', 'gold'])
    axes[0].set_title('Mean Absolute Error (Lower is Better)')
    axes[0].set_ylabel('MAE')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3)
    
    # RMSE comparison
    axes[1].bar(methods, comparison_df['RMSE'], color=['skyblue', 'lightgreen', 'salmon', 'gold'])
    axes[1].set_title('Root Mean Square Error (Lower is Better)')
    axes[1].set_ylabel('RMSE')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    # R² comparison
    axes[2].bar(methods, comparison_df['R2'], color=['skyblue', 'lightgreen', 'salmon', 'gold'])
    axes[2].set_title('R² Score (Higher is Better)')
    axes[2].set_ylabel('R²')
    axes[2].tick_params(axis='x', rotation=45)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return comparison_df

# Create comparison
comparison_results = compare_with_previous_results()

In [ ]:
# Save RAG results for further analysis
def save_rag_results(results_df, metrics):
    """Save RAG results to CSV files"""
    
    # Save detailed results
    results_df.to_csv('esg_scores_rag_analysis.csv', index=False)
    print("✓ Detailed results saved to 'esg_scores_rag_analysis.csv'")
    
    # Save evaluation metrics
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv('rag_evaluation_results.csv', index=False)
    print("✓ Evaluation metrics saved to 'rag_evaluation_results.csv'")
    
    # Save sample analyses for review
    sample_analyses = []
    for result in rag_results[:5]:  # Save first 5 detailed analyses
        sample_analyses.append({
            'Company': result['company'],
            'Predicted_Score': result['predicted_score'],
            'Benchmark_Score': result['benchmark_score'],
            'Analysis': result['analysis']
        })
    
    sample_df = pd.DataFrame(sample_analyses)
    sample_df.to_csv('rag_sample_analyses.csv', index=False)
    print("✓ Sample analyses saved to 'rag_sample_analyses.csv'")

# Save results
save_rag_results(results_df, metrics)

In [ ]:
# Analysis summary and conclusions
print("RAG-Enhanced ESG Analysis Summary")
print("=" * 50)
print(f"Companies analyzed: {len(results_df)}")
print(f"Mean Absolute Error: {metrics['MAE']:.2f}")
print(f"R² Score: {metrics['R2']:.3f}")

print("\nKey Findings:")
if metrics['MAE'] < 22:  # Better than standard LLM
    print("✓ RAG approach improved upon standard LLM performance")
else:
    print("✗ RAG approach did not improve upon standard LLM")

if metrics['MAE'] < 10:  # Comparable to BERT/BOW
    print("✓ RAG performance is competitive with traditional ML approaches")
else:
    print("✗ RAG performance still lags behind traditional ML approaches")

# Best and worst predictions
best_prediction = results_df.loc[results_df['Absolute_Error'].idxmin()]
worst_prediction = results_df.loc[results_df['Absolute_Error'].idxmax()]

print(f"\nBest prediction: {best_prediction['Company']} (Error: {best_prediction['Absolute_Error']:.1f})")
print(f"Worst prediction: {worst_prediction['Company']} (Error: {worst_prediction['Absolute_Error']:.1f})")

print("\nConclusion:")
print("The RAG approach demonstrates the potential to overcome token limitations")
print("in LLM-based ESG analysis by retrieving relevant context dynamically.")
print("Performance improvements depend on document quality and retrieval effectiveness.")